In [1]:
%load_ext autoreload
%autoreload 2
from Naver_Blog_Crawl import *
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os
import lxml
from collections import defaultdict

In [3]:
up_nb = pd.read_parquet("up_nb_241014.parquet").drop_duplicates("postUrl")

In [6]:
up_nb.loc[up_nb["nickName"].str.contains("220")].shape

(24, 19)

In [207]:

class Naver_Blog:
    base_url = "https://section.blog.naver.com/ajax/SearchList.naver"
    def __init__(self):
        pass

    def create_params(self, keyword, page = 1, size = 30, order_by = "recentdate"):
        params = {"countPerPage" : size,
                "currentPage" : page,
                "keyword" : keyword,
                "orderBy" : order_by, # sim/recentdate
                "rangeType" : "ALL",
                "startDate": None,
                "endDate": None,
                "type" : "post"}
            
        return params

    def create_headers(self, keyword):
        enc_keyword = urllib.parse.quote(keyword)
        headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
                   "Referer" : f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword={enc_keyword}",
                   "host" : "section.blog.naver.com"
                   }

        return headers

    def extract_contents(self, rq):
        json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])
        contents_list = json_rq.get("result").get("searchList")
        
        return contents_list 

    def extract_total_count(self, rq):
        json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])
        total_count = json_rq.get("result").get("totalCount")
        
        return int(total_count)

    def request_content(self, keyword, page = 1, size = 30, order_by = "recentdate"):
        params = self.create_params(keyword = keyword, page = page, size = size)
        headers = self.create_headers(keyword)
        rq = requests.get(self.base_url, params = params, headers = headers, verify = False)

        return rq
    
    @staticmethod
    def collect_tags(blog_info:str):
        log_no = blog_info.get("logNo")
        blogger_id = blog_info.get("domainIdOrBlogId")
        params_dict = {"blogId" : blogger_id,
                       "logNoList" : log_no,
                       "logType" : "mylog"}
        base_url = "https://blog.naver.com/BlogTagListInfo.naver"

        
        try:
            rq = requests.get(base_url, params = params_dict,verify = False)
            rq_json = rq.json()
            if len(rq_json["taglist"]) >= 1:
                tags = parse.unquote(rq_json["taglist"][0]["tagName"])
            else:
                tags = ""
        except Exception as e:
            print(e)
            tags = None

        return tags    

    def request_contents(self, keyword, order_by = "recentdate", max_contents_num = 1000):
        self.contents_list = []
        size = 30
        temp_rq = self.request_content(keyword, page = 1, size = 1, order_by = order_by)
        total_count = self.extract_total_count(temp_rq)
        print(total_count)
        max_contents_num = min(total_count, max_contents_num)

        page_num = max_contents_num // size
        remainder = max_contents_num % size
        
        print(f"page_num : {page_num}")
        print(f"remainder : {remainder}")

        for p in tqdm(range(100, page_num+2)):
            if p < -1:
                continue
            time.sleep(0.3)
            rq = self.request_content(keyword, page = p, size = size, order_by = order_by)
            rq_contents = self.extract_contents(rq)
            self.rq_contents = rq_contents
            
            if (p * size) > max_contents_num:
                print(p* size)
                print(max_contents_num)
                rq_contents = rq_contents[:remainder]
            
            for x in rq_contents:
                tags = self.collect_tags(blog_info = x)
                x["tags"] = tags

            self.contents_list.extend(rq_contents)
            

        return self.contents_list
    
    

In [208]:
nb = Naver_Blog()

In [209]:
up_nb_crawl = nb.request_contents(keyword = "UP가전",  max_contents_num = 5000)

50442
page_num : 166
remainder : 20


100%|██████████| 68/68 [01:55<00:00,  1.70s/it]

5010
5000


In [198]:
keyword = "UP가전"
params = nb.create_params(keyword = keyword, page = 500, size = 30)
headers = nb.create_headers(keyword)


In [199]:
base_url = "https://section.blog.naver.com/ajax/SearchList.naver"
rq = requests.get(base_url, params = params, headers = headers, verify = False)

In [202]:
rq.url

'https://section.blog.naver.com/ajax/SearchList.naver?countPerPage=30&currentPage=500&keyword=UP%EA%B0%80%EC%A0%84&orderBy=recentdate&rangeType=ALL&type=post'

In [203]:
573 * 7

4011

In [200]:
json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])

In [201]:
json_rq.get("result").get("searchList")

[{'domainIdOrBlogId': 'naek1995',
  'logNo': 223493175433,
  'gdid': '90000003_0000000000000034093B3889',
  'postUrl': 'https://blog.naver.com/naek1995/223493175433',
  'title': '2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불가의 나라를... ',
  'noTagTitle': '2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불가의 나라를 꿈꾼다',
  'contents': '모든 <strong class="search_keyword">가전</strong>제품 및 시설과 통합된 홈 IoT(Internet of Thing)의 집합체로서, 가상 환경에서 인간의 작업지식을 축적, 재구성하여 새로운 기능을... 기술개발을 위한 임상실험 규제 완화 및 의료<strong class="search_keyword">가전</strong> 보급 활성화 등 정책 지원 필요 ⑥ K-SpearShield Systems (KS3): ICT 기술접목으로 K-방산... ',
  'nickName': '한국공학한림원',
  'blogName': '한국공학한림원',
  'profileImgUrl': 'https://blogpfthumb-phinf.pstatic.net/MjAyMTA3MTNfMTc5/MDAxNjI2MTQ0NzI4MzA2.hU43nLDAsPHtSD4dXVZ43CAa4LX0GzOLm4rs31esVCwg.DoavB_hFvOHp4drIjNQN2JLODOZF2tGKXdQrbdBYyD8g.PNG.naek1995/%25EA%25B3%25B5%25ED%2595%2599%25EA%25B8%25B0%25EC%2588%25A0%25ED%2595%259C%25EB%25A6%25BC%25EC%259B%2590_SKIN%2B%25281%2529.png?type=s1',
  'addDate': 17

In [162]:
contents_list2 = nb.contents_list[:]

In [217]:
pd.DataFrame(up_nb_crawl)

,domainIdOrBlogId,logNo,gdid,postUrl,title,noTagTitle,contents,nickName,blogName,profileImgUrl,addDate,thumbnails,product,titleHead,titleHeadColor,marketPost,hasThumbnail,tags
0,seoning1004,223532551292,90000003_00000000000000340B940C7C,https://blog.naver.com/seoning1004/223532551292,삼성 냉장고 단일옵션 비교 및 분석 가이드,삼성 냉장고 단일옵션 비교 및 분석 가이드,"필수 <strong class=""search_keyword"">가전</strong>제...",seoning1004,seoning1004님의블로그,https://blogpfthumb-phinf.pstatic.net/MjAyNDA1...,1722465300000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,
1,kswkso,223531164330,90000003_00000000000000340B7EE2AA,https://blog.naver.com/kswkso/223531164330,베트남 7개월간 소비재 및 서비스 소매판매 8.7% 증가,베트남 7개월간 소비재 및 서비스 소매판매 8.7% 증가,"식품 및 식료품 판매는 10.7%, <strong class=""search_keyw...",베트남경제연구소장,ANYFIVE 베트남경제연구소,https://blogpfthumb-phinf.pstatic.net/MjAyNDAx...,1722459000000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"베트남,7개월간,소비재,서비스,소매,판매,증가,통계청,GSO"
2,vzwzwfpnt50,223532444175,90000003_00000000000000340B926A0F,https://blog.naver.com/vzwzwfpnt50/223532444175,1등 캐리어 인버터 에어컨 &amp; LG 퓨리케어 에어로타워 특<strong cl...,2024년 최대 할인! 1등 캐리어 인버터 에어컨 &amp; LG 퓨리케어 에어로타...,"온풍겸용 <strong class=""search_keyword"">UP가전</stro...",장춘자,장춘자,https://blogpfthumb-phinf.pstatic.net/MjAyNDA3...,1722447720000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,
3,abc944woo,223532321737,90000003_00000000000000340B908BC9,https://blog.naver.com/abc944woo/223532321737,내가 모르는 단어1,내가 모르는 단어1,"make <strong class=""search_keyword"">up</strong...",한우현,돌돌집사의 증권 공부,https://blogpfthumb-phinf.pstatic.net/MjAyNDA2...,1722437100000,[],None,None,None,False,False,"토익,모르는단어"
4,posmany,223532315599,90000003_00000000000000340B9073CF,https://blog.naver.com/posmany/223532315599,LG전자 휘센 오브제컬렉션 제습기 16L DQ163PECA 진짜 사용자들의 경험담,LG전자 휘센 오브제컬렉션 제습기 16L DQ163PECA 진짜 사용자들의 경험담,✔️삶의 질을 높이는 편리한 제습-당신을 위해 계속 좋아지는 가전 -LG 휘센 제습...,posmany,흘러가는 발걸음,https://blogpfthumb-phinf.pstatic.net/MjAyNDA1...,1722436800000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,naek1995,223493175433,90000003_0000000000000034093B3889,https://blog.naver.com/naek1995/223493175433,"2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","모든 <strong class=""search_keyword"">가전</strong>제...",한국공학한림원,한국공학한림원,https://blogpfthumb-phinf.pstatic.net/MjAyMTA3...,1719468240000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"한국공학한림원,IS4T포럼,한국공학한림원산업미래전략위원회,김기남회장,퍼스트무버,장석..."
1059,naek1995,223493175433,90000003_0000000000000034093B3889,https://blog.naver.com/naek1995/223493175433,"2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","모든 <strong class=""search_keyword"">가전</strong>제...",한국공학한림원,한국공학한림원,https://blogpfthumb-phinf.pstatic.net/MjAyMTA3...,1719468240000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"한국공학한림원,IS4T포럼,한국공학한림원산업미래전략위원회,김기남회장,퍼스트무버,장석..."
1060,naek1995,223493175433,90000003_0000000000000034093B3889,https://blog.naver.com/naek1995/223493175433,"2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","모든 <strong class=""search_keyword"">가전</strong>제...",한국공학한림원,한국공학한림원,https://blogpfthumb-phinf.pstatic.net/MjAyMTA3...,1719468240000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"한국공학한림원,IS4T포럼,한국공학한림원산업미래전략위원회,김기남회장,퍼스트무버,장석..."
1061,naek1995,223493175433,90000003_0000000000000034093B3889,https://blog.naver.com/naek1995/223493175433,"2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","2024년 IS4T 포럼 : 위기의 K-Industry, 미래주도 혁신과제로 대체불...","모든 <strong class=""search_keyword"">가전</strong>제...",한국공학한림원,한국공학한림원,https://blogpfthumb-phinf.pstatic.net/MjAyMTA3...,1719468240000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,Tr

In [214]:
up_nb = pd.DataFrame(contents_list1 + up_nb_crawl)
up_nb = up_nb.drop_duplicates("postUrl")

In [227]:
up_nb["addDate"].to_list()[0]

1728880800000

In [229]:
up_nb["datetime"] = pd.to_datetime(up_nb["addDate"], unit = "ms")

In [230]:
up_nb["datetime"].min()

Timestamp('2024-06-27 05:25:00')

In [231]:
up_nb.to_parquet("up_nb_241014.parquet", index = False)

In [160]:
pd.DataFrame(nb.contents_list)

,domainIdOrBlogId,logNo,gdid,postUrl,title,noTagTitle,contents,nickName,blogName,profileImgUrl,addDate,thumbnails,product,titleHead,titleHeadColor,marketPost,hasThumbnail,tags
0,glolish,223618353111,90000003_000000000000003410B147D7,https://blog.naver.com/glolish/223618353111,"Warming <strong class=""search_keyword"">Up</str...",Warming Up: 무조건 암기에 도전 - 300문장,"Warming <strong class=""search_keyword"">Up</str...",영문법틀 잡기,영문법 독학 도우미 & Big History 안내,None,1728880800000,[],None,None,None,False,False,
1,d088twgl,223618329791,90000003_000000000000003410B0ECBF,https://blog.naver.com/d088twgl/223618329791,"인기있는 LG전자 싸이킹3 싸이클론 진공청소기, K83IGY, 아이언그레이 특징","인기있는 LG전자 싸이킹3 싸이클론 진공청소기, K83IGY, 아이언그레이 특징","✅ 스타일리쉬한 아이언그레이 디자인으로 고급스러움 <strong class=""sea...",d088twgl,d088twgl님의 블로그,https://blogpfthumb-phinf.pstatic.net/MjAyNDA5...,1728879780000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"LG싸이킹,진공청소기,진공청소기추천,진공청소기후기"
2,ok-now,223618320335,90000003_000000000000003410B0C7CF,https://blog.naver.com/ok-now/223618320335,"LG전자 오브제 양문형 냉장고, 메탈 디자인으로 주방을 한층 업그레이드!","LG전자 오브제 양문형 냉장고, 메탈 디자인으로 주방을 한층 업그레이드!","다양한 기능으로 편리함 <strong class=""search_keyword"">UP...",오케이나우ㅣOK NOW,오케이나우ㅣOK NOW,https://blogpfthumb-phinf.pstatic.net/MjAyNDAy...,1728879300000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"LG전자,오브제냉장고,양문형냉장고,디오스870L,방문설치"
3,hyoni-mom,223618318791,90000003_000000000000003410B0C1C7,https://blog.naver.com/hyoni-mom/223618318791,[eme500-sp] 내돈내산 후기정보 | 경동 나비엔 숙면매트 DC 카본매트 EM...,[eme500-sp] 내돈내산 후기정보 | 경동 나비엔 숙면매트 DC 카본매트 EM...,0점 | 상품후기 3개) ■ eme500-sp 인기 후기 Top3 위 ■ 경동나비엔...,효니맘,효니맘,https://blogpfthumb-phinf.pstatic.net/MjAyNDAz...,1728879240000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,
4,4lecrkjvd,223618310174,90000003_000000000000003410B0A01E,https://blog.naver.com/4lecrkjvd/223618310174,"LG 퓨리케어 오브제컬렉션 360도 공기청정기 비교분석| AS354NS4A, AS3...","LG 퓨리케어 오브제컬렉션 360도 공기청정기 비교분석| AS354NS4A, AS3...",0점 | 상품후기 17개) 구매하러가기 ■ LG전자 퓨리케어 오브제컬렉션 360...,4lecrkjvd,핫딜투데이,https://blogpfthumb-phinf.pstatic.net/MjAyNDA5...,1728878820000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"AS354NS4A,AS354NG4A,AS334NG3C,AS204NS4A,AS334N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,hana5cjstk,223506080284,90000003_00000000000000340A00221C,https://blog.naver.com/hana5cjstk/223506080284,경동시장 금성전파사 새로고침센터 AI 세탁 체험 공간 LG 트롬하우스 팝업하우스 꼼...,경동시장 금성전파사 새로고침센터 AI 세탁 체험 공간 LG 트롬하우스 팝업하우스 꼼...,이렇게 환경을 생각하는 활동을 하다 보니 여러분들께 알려 드릴 이벤트가 생각납니다 ...,비베일리,사랑이넘치는베일리네집,https://blogpfthumb-phinf.pstatic.net/MjAyNDA0...,1720451760000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"트롬하우스,LG워시타워,LG워시콤보,금성전파사,LG트롬,LG트롬하우스,LG세탁건조기..."
3656,mitistory,223506064100,90000003_000000000000003409FFE2E4,https://blog.naver.com/mitistory/223506064100,AI 세탁 체험 공간 LG 트롬 하우스 워시콤보 팝업스토어 오픈,AI 세탁 체험 공간 LG 트롬 하우스 워시콤보 팝업스토어 오픈,"하는데요 <strong class=""search_keyword"">UP 가전</str...",미티,미티의 감성뷰™,https://blogpfthumb-phinf.pstatic.net/MjAxOTEy...,1720450740000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"LG트롬,LG트롬하우스,LG워시콤보,LG워시타워,LG세탁건조기,워시콤보,워시타워,L..."
3657,milee0822,223506037314,90000003_000000000000003409FF7A42,https://blog.naver.com/milee0822/223506037314,AI 세탁 체험 공간 LG 트롬 하우스 팝업스토어 금성전파사새로고침센터 전시 &am...,AI 세탁 체험 공간 LG 트롬 하우스 팝업스토어 금성전파사새로고침센터 전시 &am...,"<strong class=""search_keyword"">UP가전</strong> 코...",몽샘,꿈꾸는 몽샘,https://blogpfthumb-phinf.pstatic.net/MjAyNDA4...,1720449060000,[{'url': 'https://blogthumb.pstatic.net/MjAyND...,None,None,None,False,True,"LG트롬,트롬하우스,LG트롬하우스,LG워시콤보,LG워시타워,LG세탁건조기,워시콤보,..."
3658,zepirostop,223505996448,90000003_000000000000003409FEDAA0,https://blog.naver.com/zepirostop/223505996448,lg 엘지 트롬 워시타워 오브제 W20EEZ W20EEHN 모델명 트루스팀 차이 비...,lg 엘지 트롬 워시타워 오브제 W20EEZ W20EEHN 모델명 트루스팀 차이 비...,"페어링, <strong class=""search_keyword"">UP 가전</str...",흔한김군,흔한김군의 특별한 IT,https://blogpfthu

In [96]:
up_nb_crawl[0]

{'domainIdOrBlogId': 'mjmj218',
 'logNo': 223618178518,
 'gdid': '90000003_000000000000003410AE9DD6',
 'postUrl': 'https://blog.naver.com/mjmj218/223618178518',
 'title': '[이벤트안내] [삼성전자] BESPOKE AI 인덕션&amp;식기세척기 MAKEOVER STORY 화보... ',
 'noTagTitle': '[이벤트안내] [삼성전자] BESPOKE AI 인덕션&amp;식기세척기 MAKEOVER STORY 화보 소문내기 이벤트 (~10/20)',
 'contents': '<strong class="search_keyword">UP</strong> <strong class="search_keyword">UP</strong>!!) - 타 카페 공유 불가 ■필수 해시태그 #BESPOKEAI인덕션 #비스포크AI인덕션 #BESPOKEAI식기세척기 #비스포크AI식기세척기 #BESPOKEAI #AI<strong class="search_keyword">가전</strong> #BESPOKE로메이크오버화보공유이벤트_창원줌마렐라 #삼성혼수 #비스포크혼수 #세상에없던AI라이프 #삼성전자이벤트 ■당첨 경품... ',
 'nickName': '푸우콩콩',
 'blogName': '푸우콩콩',
 'profileImgUrl': 'https://blogpfthumb-phinf.pstatic.net/MjAyNDA1MjhfMjk5/MDAxNzE2ODkxMzYzNjYz.jutGFiKKuS4mrPmEw8gHHlVVuifqu2CtnikJzTRfxr4g.zIXCYR-iiGD8j4SE-SY_tnN_thvXgnU2sKQlN3_QYcEg.JPEG/image.jpg?type=s1',
 'addDate': 1728872580000,
 'thumbnails': [{'url': 'https://blogthumb.pstatic.net/MjAyNDEwMTRfMjA3/MDAxNz

In [101]:
rq = requests.get("https://blog.naver.com/BlogTagListInfo.naver?blogId=g0pwam59&logNoList=223618057836&logType=mylog", verify = False)
rq.json()[""]

{'taglist': [{'msg': '',
   'logno': '223618057836',
   'tagName': '%EC%86%8C%EC%9D%8C%EB%B0%A9%EC%A7%80EVA%EC%8A%A4%ED%8F%B0%EC%A7%80%EB%B8%94%EB%9E%99%EB%8B%A8%EB%A9%B4%EB%B0%A9%EC%9D%8C%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%B0%A9%EC%9D%8C%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%B0%A9%EC%A7%84%ED%85%8C%EC%9D%B4%ED%94%84%2C%EC%8A%A4%ED%8F%B0%EC%A7%80%ED%85%8C%EC%9D%B4%ED%94%84%2CEVA%ED%85%8C%EC%9D%B4%ED%94%84%2C%EC%86%8C%EC%9D%8C%EC%B0%A8%EB%8B%A8%2C%EC%A7%84%EB%8F%99%EA%B0%90%EC%86%8C%2C%EB%8B%A8%EB%A9%B4%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%B8%94%EB%9E%99%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%8B%A4%EC%9A%A9%EB%8F%84%ED%85%8C%EC%9D%B4%ED%94%84%2C%EA%B3%A0%ED%92%88%EC%A7%88%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%82%B4%EA%B5%AC%EC%84%B1%ED%85%8C%EC%9D%B4%ED%94%84%2C%EC%A0%80%EB%A0%B4%ED%95%9C%ED%85%8C%EC%9D%B4%ED%94%84%2C%EC%89%AC%EC%9A%B4%EC%84%A4%EC%B9%98%2C%EA%B0%95%EB%A0%A5%ED%95%9C%EC%A0%91%EC%B0%A9%EB%A0%A5%2C%ED%8F%AD30mm%EB%B0%A9%EC%9D%8C%ED%85%8C%EC%9D%B4%ED%94%84%2C%EB%91%90%EA%BB%983mm%EB%B0%A9%EC%9D%

In [119]:
collect_tags(blog_info = up_nb_crawl[5])

'태솔인테리어,태솔리모델링,연수동인테리어,연수동리모델링,연수우성1차리모델링,연수우성1차,전체리모델링,올리모델링,주방리모델링,화장실리모델링,현관중문,붙박이장,발코니시공,베란다인테리어,옥련동리모델링,청학동리모델링,선학동리모델링,송도동리모델링'

In [113]:
def collect_tags(blog_info:str):
    log_no = blog_info.get("logNo")
    blogger_id = blog_info.get("domainIdOrBlogId")
    blogger_nickname = blog_info.get("nickName")
    params_dict = {"blogId" : blogger_id,
                    "logNoList" : log_no,
                    "logType" : "mylog"}
    base_url = "https://blog.naver.com/BlogTagListInfo.naver"

    
    try:
        rq = requests.get(base_url, params = params_dict,verify = False)
        rq_json = rq.json()
        if len(rq_json["taglist"]) >= 1:
            tags = parse.unquote(rq_json["taglist"][0]["tagName"])
        else:
            tags = ""
    except Exception as e:
        print(e)
        tags = "Error"

    return tags  

In [ ]:
up_nb_crawl = pd.DataFrame(up_nb_crawl)

In [ ]:
up_nb_crawl["postUrl"].to_list()[0]

'https://blog.naver.com/lgrental_official/223618103212'

In [ ]:
with open("contents_list_240226.pkl", "wb") as f:
    pickle.dump(nb.contents_list, f)

In [ ]:
with open("contents_list_240226_2.pkl", "wb") as f:
    pickle.dump(nb.contents_list, f)

In [ ]:
result_2206 = nb.request_contents(keyword = "식기세척기",  max_contents_num = 5000)

27629


 14%|█▍        | 23/166 [00:14<01:27,  1.63it/s]


KeyboardInterrupt: 

In [ ]:
result_df = pd.DataFrame(result_2206)

In [ ]:
result_df["datetime"] = result_df["addDate"].apply(lambda x : datetime.fromtimestamp(x/1000))

In [ ]:
year_list = [2020, 2021, 2022, 2023, 2024]
result = []
for year in year_list:
    q_dict = {"Q1": (f"{year}-01-01", f"{year}-03-31"),
            "Q2": (f"{year}-04-01", f"{year}-06-31"),
            "Q3": (f"{year}-07-01", f"{year}-09-31"),
            "Q4": (f"{year}-10-01", f"{year}-12-31")}
    
    for k, v in q_dict.items():
        keyword = "미니 건조기"
        enc_keyword = urllib.parse.quote(keyword)
        base_url = "https://section.blog.naver.com/ajax/SearchList.naver"
        params = {"countPerPage" : 7,
                "currentPage" : 1,
                "keyword" : keyword,
                "orderBy" : "sim", 
                "startDate": v[0],
                "endDate": v[1],
                "type" : "post"}

        headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
                "Referer" : f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword={enc_keyword}",
                "host" : "section.blog.naver.com"
                }
        rq = requests.get(base_url, params = params, headers = headers, verify = False)
        json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])
        total_count = json_rq.get("result").get("totalCount")

        result.append({"year" : year, "Q" : k, "total_count" : total_count})
        

In [ ]:
pd.DataFrame(result)

,year,Q,total_count
0,2020,Q1,2190
1,2020,Q2,2440
2,2020,Q3,3060
3,2020,Q4,2646
4,2021,Q1,2920
5,2021,Q2,3178
6,2021,Q3,3377
7,2021,Q4,3325
8,2022,Q1,3608
9,2022,Q2,3859


In [ ]:

keyword = "건조기"
enc_keyword = urllib.parse.quote(keyword)
base_url = "https://section.blog.naver.com/ajax/SearchList.naver"
params = {"countPerPage" : 7,
          "currentPage" : 1,
          "keyword" : keyword,
          "orderBy" : "sim", 
          "startDate": "2024-01-01",
          "endDate": "2024-12-31",
          "type" : "post"}

headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
           "Referer" : f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword={enc_keyword}",
           "host" : "section.blog.naver.com"
           }
rq = requests.get(base_url, params = params, headers = headers, verify = False)
json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])
total_count = json_rq.get("result").get("totalCount")
print(total_count)

27635


In [ ]:
# 건조기
26696
28460
29763
30440
33333
32621
30969
30793
32926
33179
35406
38139
27635

38139

In [ ]:
rq = requests.get(base_url, params = params, headers = headers, verify = False)

In [ ]:
json_rq = json.loads(re.search("\{\"result\":.*", rq.text)[0])
temp = pd.DataFrame(json_rq.get("result").get("searchList"))

In [ ]:
print(temp["blogName"][0])

사진여행 청단골이야기


In [ ]:
"https://section.blog.naver.com/ajax/SearchList.naver?countPerPage=7&currentPage=1&endDate=&keyword=%EC%8B%9D%EA%B8%B0%EC%84%B8%EC%B2%99%EA%B8%B0&orderBy=sim&startDate=&type=post"

In [ ]:
keyword = ["키치", "kitsch", "모노", "컬러풀", "유니크", "미드센츄리", "클래식", "빈티지", "레트로", "모던", "심플", "럭셔리", "프리미엄", "프로페셔널", "고사양", "전문가", "미니", "소형", "네츄럴", '우드', "화이트"]
eow = ["전자 제품", "가전 제품"]

search_keywords = []
for k in keyword:
    for e in eow:
        search_keywords.append(k + " " + e)


In [ ]:
num_cores = 10
# search_keywords_split = np.array_split(search_keywords, num_cores)

In [ ]:
keyword = ["키치", "kitsch", "모노", "컬러풀", "유니크", "미드센츄리", "클래식", "빈티지", "레트로", "모던", "심플", "럭셔리", "프리미엄", "프로페셔널", "고사양", "전문가", "미니", "소형", "네츄럴", '우드', "화이트"]


In [ ]:
def collect_blog(keywords):
    naver_blog_crawl = NaverBlogCrawl()
    return naver_blog_crawl.collect_blog(keywords, keywords_start_idx = 0, maximum = 100)

In [ ]:
with Pool(num_cores) as p:
    results_async=p.map_async(collect_blog, search_keywords_split)
    results_async.get()

NameError: name 'num_cores' is not defined

In [ ]:
naver_blog_crawl.request_blog_api(keyword = "식기세척기", display=2, start=1)

{'rss': {'@version': '2.0',
  'channel': {'title': "Naver Open API - blog ::'식기세척기'",
   'link': 'https://search.naver.com',
   'description': 'Naver Search Result',
   'lastBuildDate': 'Mon, 26 Feb 2024 15:28:53 +0900',
   'total': '716698',
   'start': '1',
   'display': '2',
   'item': [{'title': '<b>식기세척기</b> 세제 믿을 수 있기에',
     'link': 'https://blog.naver.com/did4236/223359830478',
     'description': '그래서 과일 채소용 세척제로 사용이 가능하고 식품 첨가물 성분으로 젖병이랑 숟가락, 그릇 등 모든 식기까지 세척할 수 있는데요. 이렇게 성분도 착하고 믿을 만한 세제라서 그런지 <b>식기세척기</b>를 돌렸을 때 물...',
     'bloggername': '사진여행 청단골이야기',
     'bloggerlink': 'blog.naver.com/did4236',
     'postdate': '20240220'},
    {'title': '초음파<b>식기세척기</b> 사용 후 편해졌어요',
     'link': 'https://blog.naver.com/pink_rady_/223365378138',
     'description': '식기뿐만 아니지만다 여러 식재료도 초음파<b>식기세척기</b>를 토대로 씻을 수 있기 때문에 특히 맘에... 초음파<b>식기세척기</b>를 사용 할 수 보였어요. 사용이 끝이 난 주방집기와 사용 한 그릇등을 세척기에 넣은 뒤 자동...',
     'bloggername': '핑크힐병원',
     'bloggerlink': 'blog.naver.com/pink_rady_',
     'postdat

In [ ]:
naver_blog_crawl.request_blog_api(keyword = "식기세척기", display=2, start=2)

{'rss': {'@version': '2.0',
  'channel': {'title': "Naver Open API - blog ::'식기세척기'",
   'link': 'https://search.naver.com',
   'description': 'Naver Search Result',
   'lastBuildDate': 'Mon, 26 Feb 2024 15:29:08 +0900',
   'total': '716698',
   'start': '2',
   'display': '2',
   'item': [{'title': '초음파<b>식기세척기</b> 사용 후 편해졌어요',
     'link': 'https://blog.naver.com/pink_rady_/223365378138',
     'description': '식기뿐만 아니지만다 여러 식재료도 초음파<b>식기세척기</b>를 토대로 씻을 수 있기 때문에 특히 맘에... 초음파<b>식기세척기</b>를 사용 할 수 보였어요. 사용이 끝이 난 주방집기와 사용 한 그릇등을 세척기에 넣은 뒤 자동...',
     'bloggername': '핑크힐병원',
     'bloggerlink': 'blog.naver.com/pink_rady_',
     'postdate': '20240226'},
    {'title': '업소용 초음파 <b>식기세척기</b> 설거지 고민 해결',
     'link': 'https://blog.naver.com/bitsogum001/223354877343',
     'description': '최근에 업소용 초음파 <b>식기세척기</b> 무이자 할부로 구매했는데 그동안의 걱정 거리가 싹 사라졌어요.... 인건비 문제로 고민을 이어나가다가 우연히 주변 사장님과 대화를 하게 됐는데 업소용 초음파 <b>식기세척기</b>...',
     'bloggername': '계룡산 빛소금 체험관',
     'bloggerlink': 'blog.naver.com/bitsogum

In [ ]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(["식기세척기"], 
                                        keywords_start_idx = 0, num_blogs=1, maximum = 1)
# pd.DataFrame(result).to_excel("temp.xlsx")

In [ ]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(["식기세척기"], 
                                        keywords_start_idx = 0, num_blogs=100, maximum = 100000)
# pd.DataFrame(result).to_excel("temp.xlsx")

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(result).to_parquet("dw_naver_blog_240226.parquet", index = False)

https://www.revu.net/campaign/img.php?p=951a0b94258ddbcd84b2218fb9ac56644bbdf63758ac2e2474d30cfbca8a87b9&v=4
https://www.revu.net/campaign/img.php?p=40b16e63d439bffdef8dc4fa06625fc1b13346f4d60c5bc6a14c3429454c0258&v=4

In [ ]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(search_keywords[:4], keywords_start_idx = 0, num_blogs= 1)
pd.DataFrame(result).to_excel("temp.xlsx")

In [ ]:
naver_blog_crawl = NaverBlogCrawl() # 23.8
result = naver_blog_crawl.collect_blog("LG 공기청정기")

0, keyword : LG 공기청정기, # of blogs : 196


In [ ]:
import pickle
with open("blog_dict/LG 공기청정기_blog_dict.pkl", "rb") as f:
    lg_ap_blog = pickle.load(f)

In [ ]:
pd.DataFrame(lg_ap_blog).to_excel("lg_ap_blog.xlsx", index= False)

In [ ]:
def merge_dict(org_dict:dict, new_dict:dict, type:str = "full"):
    if type == "full":
        keys = pd.unique(list(org_dict.keys()) + list(new_dict.keys()))
    elif type == "left":
        keys = list(org_dict.keys())
    elif type == "right":
        keys = list(new_dict.keys())


    for key in keys:
        if isinstance(new_dict[key], list):
            org_dict[key].extend(new_dict[key])
        else:
            org_dict[key].append(new_dict[key])
    
    return org_dict

In [ ]:
result_dict = defaultdict(list)
blog_dicts = os.listdir("./blog_dict")
for x in blog_dicts:
    with open(f"./blog_dict/{x}", "rb") as f:
        blog_dict = pickle.load(f)
        result_dict = merge_dict(result_dict, blog_dict)

In [ ]:
result_df = pd.DataFrame(result_dict)

In [ ]:
result_df.to_excel("메디큐브블로그.xlsx")

In [ ]:
result_df["keyword"].drop_duplicates()

In [ ]:
duplicated_blogs = result_df[["writer_name", "post_date"]].value_counts()
duplicated_blogs = duplicated_blogs.loc[duplicated_blogs>1].reset_index()

In [ ]:
result_df = result_df.loc[result_df["writer_name"].isin(duplicated_blogs["writer_name"])==False]

In [ ]:
def save_images(img_urls:str, img_names:str, img_dir:str = "./image"):
    
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)


    for img_url, img_name in zip(img_urls, img_names):
        # image 저장        
        try:
            img_rq = requests.get(img_url, verify = False)
            with open(f"{img_dir}/{img_name.replace('/','_')}", "wb") as f:
                f.write(img_rq.content)
        except: 
            print(f"img_name : {img_name}, img_url : {img_url}")

In [ ]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수
        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key
        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
#     tqdm.pandas()
    #func = partial(func, **params)
    
    # def map_func(data):
    #     return data.apply(func, axis = 1, **params)
#         return data.progress_apply(func, axis = 1, **params)
        
        
    with Pool(num_cores) as p:
        p.map(func, df_list)
#         pd_result = p.starmap(func, **params)

In [ ]:
def save_images_par(temp_df):
    for row in temp_df[["keyword", "images", "image_urls"]].iterrows():
        save_images(img_urls = row[1][2], img_names = row[1][1], img_dir = f"/mnt/d/Crawl/Naver_Blog_Crawl/sample_image/{row[1][0].replace('/','_')}")

In [ ]:
parallelize_dataframe(df = result_df, func = save_images_par, group_keys = "keyword", num_cores = 40)

In [ ]:
result_df = result_df.loc[result_df["writer_name"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|
                                                                   (센터)|(중고)|(청소)|(가전)|(렌탈)|(베스트샵)|(삼성)|
                                                                   (리폼)|(교체)|(홈쇼핑)|(닷컴)|(공식)|(\(주\))|(특가)|(이벤트)|(할인)|
                                                                   (신문)|(선택장애 살아가기)|(삼시육끼)""")==False]
result_df = result_df.loc[result_df["title"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|
                                                             (중고)|(청소)|(가전)|(렌탈)|(베스트샵)|(삼성)|(리폼)|(교체)|
                                                             (체험단)|(홈쇼핑)|(닷컴)|(기부)|(방송)|(특가)|(이벤트)|(할인)|
                                                             (리스트)|(요약)|(리포트)""")==False]
result_df = result_df.loc[result_df["tags"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|(중고)|(청소)|
                                                            (가전)|(렌탈)|(베스트샵)|(삼성)|(리폼)|(교체)|(체험단)|(홈쇼핑)|(닷컴)|
                                                            (특가)|(이벤트)|(할인)""")==False]

/tmp/ipykernel_13533/4030479231.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["writer_name"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|
/tmp/ipykernel_13533/4030479231.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["title"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|
/tmp/ipykernel_13533/4030479231.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["tags"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|(중고)|(청소)|


In [ ]:
result_df = result_df.loc[result_df["post_date"]>="2020000"]

In [ ]:
result_df = result_df.loc[result_df["contents"].str.contains("""(랭킹)|(상세 페이지)|(체험단)|(입금)|(익일)|(검색)|(프로모션)|
                                                                (소식)|(포토툰)|(설치)|(닷컴)|(주소)|(전화)|(영업시간)|(스토어)|
                                                                ([\d]+원)|(010-)|(주식)|(카카오톡)|(채널)|(주가)|(주주)|(차트)|(전시)|
                                                                (가정의)|([\d]{3,4}-[\d]{3,4}-[\d]{4})|(이벤트)|(바로가기)|(도서)|(리스트)|
                                                                (요약)|(리포트)|(클릭)|(할\.인)|(구\.매)|(전문점)|(교환)|(소정)|(기자)|(출처)|
                                                                (소공미디어)|(정시)|(접수)|(참여)|(관련주)|(매수)""")==False]

/tmp/ipykernel_13533/2667990024.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["contents"].str.contains("""(랭킹)|(상세 페이지)|(체험단)|(입금)|(익일)|(검색)|(프로모션)|


In [ ]:
result_df = result_df.loc[(result_df["image_urls"].apply(lambda x: len(x)) != 0)]

In [ ]:
result_df = result_df.loc[result_df["image_urls"].apply(lambda x: len(x)) < 30]

In [ ]:
result_df.to_excel("blog_data_230111.xlsx", index = False, encoding = "CP949")

/home/seho/.local/lib/python3.10/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
blogId = re.search(r"blogId=([a-zA-Z0-9_-]+)", blog_url).group(1)
logNoList = re.search(r"logNo=([\d]+)", blog_url).group(1)

In [ ]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수
        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key
        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
#     tqdm.pandas()
    #func = partial(func, **params)
    
    # def map_func(data):
    #     return data.apply(func, axis = 1, **params)
#         return data.progress_apply(func, axis = 1, **params)
        
        
    with Pool(num_cores) as p:
        pd_result = pd.concat(p.map(func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [ ]:
def get_blog_like_count(blog_url):
    blogId = re.search(r"blogId=([a-zA-Z0-9_-]+)", blog_url).group(1)
    logNoList = re.search(r"logNo=([\d]+)", blog_url).group(1)

    like_base_url = "https://blog.like.naver.com/v1/search/contents"
    params_dict = {"suppress_response_codes" : "true",
                   "q" : f"BLOG[{blogId}_{logNoList}]"}

    blog_rq = requests.get(like_base_url, params_dict)
    
    if len(blog_rq.json()["contents"][0]["reactions"]) == 0:
        like_count = 0
    else:
        like_count = blog_rq.json()["contents"][0]["reactions"][0]["count"]
    return like_count


In [ ]:
def get_like_par(url_list):
    like_list = []
    blog_url_list = []
    for i, blog_url in enumerate(url_list):
        blog_url_list.append(blog_url)
        like_list.append(get_blog_like_count(blog_url))

    return pd.DataFrame({"blog_url" : blog_url_list, "like" : like_list})

In [ ]:
like_df = parallelize_dataframe(df = result_df["blog_url"], func = get_like_par, num_cores = 50)

In [ ]:
result_df = pd.merge(result_df, like_df, on = "blog_url")
print(result_df.shape)

(18431, 11)


In [ ]:
result_df = pd.merge(result_df, product_data[["keyword", "category1Name", "category2Name", "category3Name", "category4Name"]].drop_duplicates("keyword"), on = "keyword")
print(result_df.shape)

(18431, 15)


In [ ]:
result_df = result_df.loc[result_df["category2Name"] != "인테리어소품"]

In [ ]:
category3_df = pd.read_csv("소카테고리.csv", encoding = "CP949")

In [ ]:
result_df = pd.merge(result_df, category3_df, left_on = "category3Name", right_on = "소카테고리")

In [ ]:
virtual_user_df = pd.read_excel("~/Project_C/1차오픈용계정_0428.xlsx")

In [ ]:
result_df["알파카테고리"].unique()

array(['요리/주방', '음향/사운드', '영상가전', '생활가전', '뷰티가전', '계절가전', '건강가전', '의류관리기',
       '위생가전'], dtype=object)

In [ ]:
alpha_category = ["음향/사운드", "계절가전", "영상가전", "요리/주방", "생활가전", "뷰티가전", "위생가전", "의류관리기", "건강가전"]

In [ ]:
category_num = (virtual_user_df[alpha_category].isna() == False).sum().to_dict()

In [ ]:
category_num

{'음향/사운드': 20,
 '계절가전': 40,
 '영상가전': 35,
 '요리/주방': 50,
 '생활가전': 60,
 '뷰티가전': 25,
 '위생가전': 15,
 '의류관리기': 10,
 '건강가전': 10}

In [ ]:
sample_result_df = result_df.groupby("알파카테고리").apply(lambda x: x.sort_values("like", ascending = False).head(category_num[x["알파카테고리"].to_list()[0]]*5)).reset_index(drop=True)
sample_result_df["id"] = [f"blog_{i+1}" for i in range(sample_result_df.shape[0])]

In [ ]:
sample_result_df.shape

(1325, 18)

In [ ]:
import random

In [ ]:
virtual_user_df["blog_list"] = [[] for _ in range(virtual_user_df.shape[0])]

In [ ]:
for category in alpha_category:
    category_user = virtual_user_df.loc[virtual_user_df[category] == "O"]
    # num_user = category_user.shape[0]
    user_ids = category_user["번호"]
    
    sampled_list = []
    for user_id in user_ids:
        category_blog_id = sample_result_df.loc[(sample_result_df["알파카테고리"] == category) & (sample_result_df["id"].isin(sampled_list) == False), "id"].to_list()
        temp_sampled_list = random.sample(category_blog_id, 5)
        sampled_list.extend(temp_sampled_list)

        temp = virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"]
        temp[temp.index.item()].extend(temp_sampled_list)

In [ ]:
virtual_user_df

,번호,이메일(@test.com),닉네임,비밀번호,음향/사운드,계절가전,영상가전,요리/주방,생활가전,뷰티가전,위생가전,의류관리기,건강가전,blog_list
0,1,code1@test.com,ellys.home,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1249, blog_1242, blog_1272, blog_1263, b..."
1,2,code2@test.com,cosydaily,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1190, blog_1187, blog_1260, blog_1218, b..."
2,3,code3@test.com,applely88,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1235, blog_1203, blog_1258, blog_1196, b..."
3,4,code4@test.com,jshpink82,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1216, blog_1257, blog_1214, blog_1230, b..."
4,5,code5@test.com,zooty7979,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1177, blog_1253, blog_1191, blog_1239, b..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,code146@test.com,wyatt_home,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_930, blog_975, blog_1070, blog_1049, blo..."
146,147,code147@test.com,yellow_8,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_987, blog_1076, blog_967, blog_1087, blo..."
147,148,code148@test.com,y3ll0w,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_882, blog_891, blog_973, blog_1028, blog..."
148,149,code149@test.com,for_yellow,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_864, blog_871, blog_1040, blog_883, blog..."


In [ ]:
virtual_user_df.to_excel("가상유저_블로그_id_220502.xlsx")

In [ ]:
sample_result_df.to_excel("sample_naver_blog_220502.xlsx", index = False)

In [ ]:
parallelize_dataframe(df = sample_result_df, func = save_images_par, group_keys = "keyword", num_cores = 60)

img_name : LG전자 43NANO75KPA_생활속의 이득이 되는 정보를 수집하는 푸름나무_20210621_img_9.jpg, img_url : https://thisiswhymymomsmiled.com/wp-content/uploads/2021/06/20210621_191034-300x18.png
img_name : 로보락 S7_은`s 기쁜하루_20220226_img_17.jpg, img_url : https://tble.kr/camimg.php?tcode=MjIyNjQzLW5pZF8yMzk4NzQyMS0yMzUxMDcz
img_name : 로보락 S7_Bling Blossom_20211217_img_10.jpg, img_url : https://tble.kr/camimg.php?tcode=MjA1MDQwLW5pZF85MzA0ODMwNC0yMDg2ODg4


In [ ]:
pd.DataFrame(result_dict).to_parquet("blog_data_220312.parquet", index = False)

In [ ]:
pd.DataFrame(result_dict).to_excel("blog_data_220501.xlsx", index = False, encoding = "CP949")

In [ ]:
virtual_user_df.to_excel("가상유저_블로그_id_220502.xlsx")

In [ ]:
virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"].index == 1

array([ True])

In [ ]:
virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"]

In [ ]:
sample_result_df.loc[sample_result_df["알파카테고리"] == category, "id"]

300    blog_301
301    blog_302
302    blog_303
303    blog_304
304    blog_305
         ...   
535    blog_536
536    blog_537
537    blog_538
538    blog_539
539    blog_540
Name: id, Length: 240, dtype: object

In [ ]:
category_user = virtual_user_df.loc[virtual_user_df["음향/사운드"] == "O"]
num_user = category_user.shape[0]

In [ ]:
naver_blog_crawl = NaverBlogCrawl()

In [ ]:
up_nbc = naver_blog_crawl.collect_blog(keywords = "UP가전", keywords_start_idx = 0, num_blogs = 10000, maximum = 10000, executor = None, image = False)

In [ ]:
naver_blog_crawl.request_blog_api(keyword= "UP가전", display = 100, start = 2)

{'rss': {'@version': '2.0',
  'channel': {'title': "Naver Open API - blog ::'UP가전'",
   'link': 'https://search.naver.com',
   'description': 'Naver Search Result',
   'lastBuildDate': 'Mon, 14 Oct 2024 10:21:32 +0900',
   'total': '53208',
   'start': '2',
   'display': '100',
   'item': [{'title': 'LG 공기청정기 업그레이드 <b>UP가전</b> 업가전기능 알려드림',
     'link': 'https://blog.naver.com/cpss1656/223617531519',
     'description': '기대되는 LG <b>UP가전</b> 사용기 220 코드앤코드 앱으로 오시면 찐 사용 유저들의 사용기를 볼수가 있어요 #LG전자 #LGUP가전 #<b>UP가전</b> #업가전 #<b>UP가전</b>센터 #UP기능 #LG업그레이드기능 #업그레이드<b>UP가전</b> #220<b>UP가전</b>',
     'bloggername': '쓰리신의 개인취향',
     'bloggerlink': 'blog.naver.com/cpss1656',
     'postdate': '20241013'},
    {'title': 'LG <b>UP가전</b> 이제는 리빙가전도 진화한다!',
     'link': 'https://blog.naver.com/hongya931/222941612716',
     'description': '지난번 연말 파티를 위해 놀러 갔다 키친 LG <b>UP가전</b> 야무지게 써보고 온 후기 잘 보셨나요? 아직 못... 리빙 가전의 경우 핸드폰만큼 빠르게 바꾸지 않기도 하고, 지속적인 업데이트가 되지 않아 아쉽지만...',
     'bloggername': '썬데이 홍야라이프(Feat.짜몽&팡)',